In [1]:
import numpy as np
from dataclasses import dataclass, field
from typing import List, Tuple, Set, Dict
from collections import defaultdict

from grid import Point, Net, Grid

In [2]:
@dataclass
class TraceCandidate:
    start: Point
    end: Point
    path: Set[Point] # all points in the trace
    net_id: str
    probability: float

In [3]:
def generate_all_traces(traces: List[TraceCandidate]) -> Dict[Tuple[Point, Point], List[TraceCandidate]]:
    traces_by_prob = defaultdict(list)

    for trace in traces:
        key = (trace.start, trace.end)
        traces_by_prob[key].append(trace)
        
    for pair in traces_by_prob:
        traces_by_prob[pair].sort(key=lambda t: t.probability, reverse=True)

    return traces_by_prob

In [26]:
pa = Point(1, 1)
pb = Point(4, 5)

pa1 = Point(1, 2)
pa2 = Point(1, 3)
pa3 = Point(1, 4)
pa4 = Point(2, 5)
pa5 = Point(3, 5)
pa6 = Point(2, 3)
pa7 = Point(3, 4)

trace_ab1 = TraceCandidate(start=pa, end=pb, path={pa, pa1, pa2, pa3, pa4, pa5, pb}, net_id='net1', probability=0.6)
trace_ab2 = TraceCandidate(start=pa, end=pb, path={pa, pa6, pa7, pb}, net_id='net1', probability=0.8)


pc = Point(4, 1)
pd = Point(1, 5)

pc1 = Point(2, 4)
pc2 = Point(3, 3)
pc3 = Point(4, 2)



trace_cd1 = TraceCandidate(start=pc, end=pd, path={pc, pc1, pc2, pc3}, net_id='net2', probability=0.5)

traces = [trace_ab1, trace_ab2, trace_cd1]

grouped = generate_all_traces(traces)

for key, val in grouped.items():
    print(f"\n From {key[0]} to {key[1]}:")
    for t in val:
        print(f"  prob={t.probability}, path={t.path}")


 From Point(x=1, y=1) to Point(x=4, y=5):
  prob=0.8, path={Point(x=2, y=3), Point(x=4, y=5), Point(x=1, y=1), Point(x=3, y=4)}
  prob=0.6, path={Point(x=1, y=2), Point(x=1, y=1), Point(x=1, y=4), Point(x=4, y=5), Point(x=2, y=5), Point(x=1, y=3), Point(x=3, y=5)}

 From Point(x=4, y=1) to Point(x=1, y=5):
  prob=0.5, path={Point(x=2, y=4), Point(x=3, y=3), Point(x=4, y=1), Point(x=4, y=2)}


In [5]:
def cross_conflict(path1: Set[Point], path2: Set[Point]) -> bool:
    for p in path1:
        # 检查对角方向是否存在交叉
        diag1 = Point(p.x + 1, p.y + 1)
        if diag1 in path1 and Point(p.x, p.y + 1) in path2 and Point(p.x + 1, p.y) in path2:
            return True
        diag2 = Point(p.x + 1, p.y - 1)
        if diag2 in path1 and Point(p.x, p.y - 1) in path2 and Point(p.x + 1, p.y) in path2:
            return True
    return False

In [24]:
def backtrack_traces(traces_by_prob: Dict[Tuple[Point, Point], List[TraceCandidate]]) -> Dict[Tuple[Point, Point], List[TraceCandidate]]:
    
    result = []
    #occupied = set()
    keys = list(traces_by_prob.keys())
    
    def backtrack(index) -> bool:
        if index == len(keys):
            return True
        
        key = keys[index]
        candidates = traces_by_prob[key] # candidates for current pair
        
        for trace in candidates:
            
            conflict = False
            
            for other_key, other_trace in result:
                # 如果是同一个 net_id，直接跳过检测
                if trace.net_id == other_trace.net_id:
                    continue

                # 判断是否有路径重合
                if trace.path & other_trace.path:
                    conflict = True
                    break

                # 判断是否斜对角冲突
                if cross_conflict(trace.path, other_trace.path):
                    conflict = True
                    break

            if conflict:
                continue
            else: 
                result.append((key, trace))
                #occupied.update(trace.path)
                
            
            if backtrack(index + 1):
                return True
            else:
                result.pop()
                #occupied.difference_update(trace.path)
            
        return False 
    
    success = backtrack(0)
    if success:
        return dict(result)  # 返回 dict[(start, end)] = trace
    else:
        print("No available traces!")
        return None

In [27]:
selected_traces = backtrack_traces(grouped)
print("\n")
print("\n")
from pprint import pprint

pprint(selected_traces)

print("\n")
print("\n")

for key, trace in selected_traces.items():
    print(f"From {key[0]} to {key[1]}:")
    print(f"  Path: {trace.path}")
    print(f"  Probability: {trace.probability}")
    print()





{(Point(x=1, y=1), Point(x=4, y=5)): TraceCandidate(start=Point(x=1, y=1), end=Point(x=4, y=5), path={Point(x=1, y=2), Point(x=1, y=1), Point(x=1, y=4), Point(x=4, y=5), Point(x=2, y=5), Point(x=1, y=3), Point(x=3, y=5)}, net_id='net1', probability=0.6),
 (Point(x=4, y=1), Point(x=1, y=5)): TraceCandidate(start=Point(x=4, y=1), end=Point(x=1, y=5), path={Point(x=2, y=4), Point(x=3, y=3), Point(x=4, y=1), Point(x=4, y=2)}, net_id='net2', probability=0.5)}




From Point(x=1, y=1) to Point(x=4, y=5):
  Path: {Point(x=1, y=2), Point(x=1, y=1), Point(x=1, y=4), Point(x=4, y=5), Point(x=2, y=5), Point(x=1, y=3), Point(x=3, y=5)}
  Probability: 0.6

From Point(x=4, y=1) to Point(x=1, y=5):
  Path: {Point(x=2, y=4), Point(x=3, y=3), Point(x=4, y=1), Point(x=4, y=2)}
  Probability: 0.5

